In [1]:
!pip install sentence-transformers faiss-cpu --quiet



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss


c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("../data/large_dataset.csv.txt")

# Prepare one combined text field
df["email_text"] = df["subject"].fillna("") + " " + df["body"].fillna("")
df = df[["email_text", "tag"]]  # optional keep tag for debugging

df.head()


,email_text,tag
0,Unable to access shared mailbox I am getting a...,access_issue
1,Rule not triggering Our auto-assignment rule i...,workflow_issue
2,Email threads not merging Two replies from the...,threading_issue
3,Tag suggestions incorrect Tag suggestions are ...,tagging_accuracy
4,Drafts disappearing Draft replies disappear wh...,ui_bug


In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")
print("Embedding model loaded!")


c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

Embedding model loaded!


In [6]:
corpus = df["email_text"].tolist()
corpus_embeddings = model.encode(corpus, convert_to_numpy=True, normalize_embeddings=True)

corpus_embeddings.shape


(60, 384)

In [7]:
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # inner product = cosine if normalized
index.add(corpus_embeddings)
print("FAISS index created!")


FAISS index created!


In [8]:
def search(query, top_k=3):
    # Create embedding
    query_embedding = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

    # Search
    scores, indices = index.search(query_embedding, top_k)

    results = []
    for idx, score in zip(indices[0], scores[0]):
        results.append({
            "email": corpus[idx],
            "score": float(score),
            "tag": df.iloc[idx]["tag"]
        })
    return results


In [9]:
query = "automation rule not working"
results = search(query, top_k=3)
results


[{'email': 'Rule not triggering Our auto-assignment rule is no longer firing for emails with the word Refund.',
  'score': 0.4507540166378021,
  'tag': 'workflow_issue'},
 {'email': "Rules not saving Workflow rules don't save after clicking Submit.",
  'score': 0.434370756149292,
  'tag': 'workflow_bug'},
 {'email': 'Automation delay Our automation to mark emails as pending is taking 2–3 minutes to run.',
  'score': 0.40733572840690613,
  'tag': 'automation_delay'}]

In [10]:
def rag_answer(query):
    matches = search(query, top_k=3)

    top = matches[0]
    confidence = (top["score"] + 1) / 2  # normalize 0–1

    reasoning = f"The top matched email mentions similar context: '{top['email'][:80]}...'"

    return {
        "query": query,
        "top_result": top,
        "confidence": round(confidence, 3),
        "reasoning": reasoning,
        "alternates": matches[1:]
    }

rag_answer("unable to assign emails automatically")


{'query': 'unable to assign emails automatically',
 'top_result': {'email': 'Auto-assign slow Incoming emails remain unassigned for up to 2 minutes.',
  'score': 0.6926428079605103,
  'tag': 'automation_delay'},
 'confidence': 0.846,
 'reasoning': "The top matched email mentions similar context: 'Auto-assign slow Incoming emails remain unassigned for up to 2 minutes....'",
 'alternates': [{'email': 'Assignee reset Emails revert to unassigned randomly.',
   'score': 0.665044367313385,
   'tag': 'assignment_issue'},
  {'email': 'Incorrect user assignments Emails are being assigned to the wrong agent.',
   'score': 0.6207638382911682,
   'tag': 'assignment_bug'}]}